# Differentiable Uncalibrated Imaging

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torchkbnufft as tkbn
from skimage.data import shepp_logan_phantom
from skimage.transform import resize

In [ ]:
def init_phantom(size=64):
    image = shepp_logan_phantom().astype(np.float32)  # Shape format (H,W)
    image = resize(image, (size, size), anti_aliasing=True)
    image = torch.tensor(image, dtype=torch.float)
    image = (image + 1j * torch.zeros_like(image)).unsqueeze(0)
    return image

def init_meas_coords(size=64, accel=2):
    meas_coords = torch.rand(2, int(size * size / accel)) * 2 * np.pi - np.pi
    return meas_coords

def imshow(ndarray, ax=None, title=''):
    show = False
    if ax is None:
        show = True
        ax = plt.gca()
    ax.imshow(np.squeeze(ndarray), cmap='gray')
    ax.set_axis_off()
    ax.set_title(title)
    if show: plt.show()

In [ ]:
class Measurement(torch.nn.Module):
    
    """ Measurement representation model """
    
    def __init__(self):
        pass

    def forward(self):
        pass

class ForwardOp:
    def __init__(self, meas_coords):  
        self.meas_coords = meas_coords
        self.interp_matrices = tkbn.calc_tensor_spmatrix(meas_coords, im_size=(64,64))
        self.nufft = tkbn.KbNufft(im_size=(64,64))
        self.inufft = tkbn.KbNufftAdjoint(im_size=(64,64))
    def __call__(self, image): return self.nufft(image, self.meas_coords)
    def H(self, kdata):        return self.inufft(kdata, self.meas_coords, self.interp_mats)

class Reconstructor:
    def __init__(self):
        pass
    def __call__(self):
        pass

In [176]:
def loss_fitting():
    pass

def loss_consistency():
    pass

def compute_loss():
    pass

In [ ]:
image_gt = init_phantom()
meas_coords = init_meas_coords(accel=2)

In [ ]:
R = Measurement()
A = ForwardOp(meas_coords)
G = Reconstructor()

meas_pred = A(image_gt)
print(meas_coords.shape)


torch.Size([2, 2048])
